In [31]:
import matplotlib
import numpy as np
matplotlib.use('Agg')
from keras.regularizers import l2
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, MaxoutDense, Reshape, Lambda, Input
from keras.layers import Convolution2D, MaxPooling2D
from matplotlib import pyplot as plt
from data_utils import data_utils
import keras.backend as K

In [67]:
img_rows = 24
img_cols = 24
model = Sequential()
model.add(Convolution2D(32, 3, 3, input_shape=(3, img_rows, img_cols), trainable=True, W_regularizer=l2(0.01)))

def maxout(x):
    out = None
    for i in range(0, 32):
        max = K.max(x[:, i * 2:(i + 1) * 2, :, :], axis=1)
        if out == None:
            out = max
        else:
            out = K.concatenate([out, max], axis=1)
    return out

def maxout_output_shape(input_shape):
    shape = list(input_shape)
    shape[1] = 16
    return tuple(shape)

model.add(Lambda(function=maxout, output_shape=maxout_output_shape))
model.compile(optimizer='adam', loss="mse")
X = np.ones((1, 1, 24, 24))
print(X.shape)
get_3rd_layer_output = K.function([model.layers[0].input],
                                  [model.layers[1].output])

layer_output = get_3rd_layer_output([X])

(1, 1, 24, 24)


ValueError: CorrMM images and kernel must have the same stack size

Apply node that caused the error: CorrMM{valid, (1, 1)}(convolution2d_input_50, Subtensor{::, ::, ::int64, ::int64}.0)
Toposort index: 2
Inputs types: [TensorType(float32, 4D), TensorType(float32, 4D)]
Inputs shapes: [(1, 1, 24, 24), (32, 3, 3, 3)]
Inputs strides: [(2304, 2304, 96, 4), (108, 36, -12, -4)]
Inputs values: ['not shown', 'not shown']
Outputs clients: [[Elemwise{Add}[(0, 0)](CorrMM{valid, (1, 1)}.0, Reshape{4}.0)]]

Backtrace when the node is created(use Theano flag traceback.limit=N to make it longer):
  File "<ipython-input-67-b555a4718732>", line 4, in <module>
    model.add(Convolution2D(32, 3, 3, input_shape=(3, img_rows, img_cols), trainable=True, W_regularizer=l2(0.01)))
  File "/usr/local/lib/python3.4/dist-packages/keras/models.py", line 114, in add
    layer.create_input_layer(batch_input_shape, input_dtype)
  File "/usr/local/lib/python3.4/dist-packages/keras/engine/topology.py", line 341, in cre

In [64]:
X = np.ones((1, 1, 24, 24))
model = Sequential()

model.add(Convolution2D(32, 3, 3, input_shape=(1, 24, 24), trainable=True, W_regularizer=l2(0.01)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(32, 3, 3, trainable=True, W_regularizer=l2(0.01)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(32, 3, 3, trainable=True, W_regularizer=l2(0.01)))
model.compile(optimizer='adam', loss="mse")

In [52]:
inputs = Input(shape=(784,))

# a layer instance is callable on a tensor, and returns a tensor
x = Dense(64, activation='relu')(inputs)
x = Dense(64, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)

# this creates a model that includes
# the Input layer and three Dense layers
model = Model(input=inputs, output=predictions)
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [65]:
get_3rd_layer_output = K.function([model.layers[0].input],
                                  [model.layers[1].output])
layer_output = get_3rd_layer_output([X])
layer_output

ValueError: Input of CAReduce{maximum} has zero-size on axis 1
Apply node that caused the error: Reduce{maximum}{1}(Subtensor{::, int64:int64:}.0)
Toposort index: 51
Inputs types: [TensorType(float32, 4D)]
Inputs shapes: [(1, 0, 22, 22)]
Inputs strides: [(61952, 1936, 88, 4)]
Inputs values: [array([], shape=(1, 0, 22, 22), dtype=float32)]
Outputs clients: [[Join(TensorConstant{1}, Join.0, max)]]

HINT: Re-running with most Theano optimization disabled could give you a back-trace of when this node was created. This can be done with by setting the Theano flag 'optimizer=fast_compile'. If that does not work, Theano optimizations can be disabled with 'optimizer=None'.
HINT: Use the Theano flag 'exception_verbosity=high' for a debugprint and storage map footprint of this apply node.